# Transformações

Realizar transformações nos datasets, de modo a converter o dado ingestado previamente no formato mais otimizado para Big Data, o formato parquet, particionando-o fisicamente quando necessário.

## Transformação das tabelas de formato JSON para Parquet

### Definindo um schema para cada tabela

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType

In [ ]:
# schema da tabela pais

schema_pais = StructType([
    StructField("ID", IntegerType(), True),
    StructField("NOME", StringType(), True),
    StructField("SIGLA", StringType(), True),
    StructField("SLUG", StringType(), True)
])

In [ ]:
# schema da tabela dados_paises

schema_dados_paises = StructType([
    StructField("ACTIVE", IntegerType(), True),
    StructField("CONFIRMED", IntegerType(), True),
    StructField("DATE", DateType(), True),
    StructField("DEATHS", IntegerType(), True),
    StructField("ID", IntegerType(), True),
    StructField("ID_PAIS", IntegerType(), True),
    StructField("LAT", IntegerType(), True),
    StructField("LON", IntegerType(), True),
    StructField("RECOVERED", IntegerType(), True),
])

In [ ]:
# schema da tabela summary

schema_summary = StructType([
    StructField("ID", IntegerType(), True),
    StructField("ID_PAIS", IntegerType(), True),
    StructField("NEWCONFIRMED", IntegerType(), True),
    StructField("TOTALCONFIRMED", IntegerType(), True),
    StructField("NEWDEATHS", IntegerType(), True),
    StructField("TOTALDEATHS", IntegerType(), True),
    StructField("NEWRECOVERED", IntegerType(), True),
    StructField("TOTALRECOVERED", IntegerType(), True),
    StructField("DATE", DateType(), True),
])

In [ ]:
# Leitura dos arquivos que estão no diretório _raw

df_summary_json = spark.read.json("dbfs:/FileStore/_covid_data_lake/_raw/_summary/summary_paises.json", schema=schema_summary)
df_pais_json = spark.read.json("dbfs:/FileStore/_covid_data_lake/_raw/_pais/pais.json", schema=schema_pais)
df_dados_paises_json = spark.read.json("dbfs:/FileStore/_covid_data_lake/_raw/_dados_paises/dados_paises.json", schema=schema_dados_paises)

In [ ]:
# Salvando arquivos com formato Parquet no diretório _ready

df_summary_json.write.parquet("dbfs:/FileStore/_covid_data_lake/_ready/_summary/summary_paises.parquet", mode='overwrite')
df_pais_json.write.parquet("dbfs:/FileStore/_covid_data_lake/_ready/_pais/pais.parquet", mode='overwrite')
df_dados_paises_json.write.parquet("dbfs:/FileStore/_covid_data_lake/_ready/_dados_paises/dados_paises.parquet", mode='append')

## Carregando o dataset summary_paises, pais e dados_paises

In [ ]:
data_pais = "dbfs:/FileStore/_covid_data_lake/_ready/_pais/pais.parquet"
data_dados_paises = "dbfs:/FileStore/_covid_data_lake/_ready/_dados_paises/dados_paises.parquet"
data_summary_paises = "dbfs:/FileStore/_covid_data_lake/_ready/_summary/summary_paises.parquet"

In [ ]:
df_pais = spark.read.load(data_pais, format='parquet', header='true')
display(df_pais)
df_pais.printSchema()

In [ ]:
df_dados_paises = spark.read.load(data_dados_paises, format='parquet', header='true')
display(df_dados_paises)
df_dados_paises.printSchema()

In [ ]:
df_summary_paises = spark.read.load(data_summary_paises, format='parquet', header='true')
display(df_summary_paises)
df_summary_paises.printSchema()

## Criação de tabelas base para análise de dados

### Tabela summary_country

In [ ]:
# Realizado a junção das tabelas paises e resumo
df_summary_dados_pais = df_pais.join(df_summary_paises,on=df_pais.ID == df_summary_paises.ID_PAIS)
display(df_summary_dados_pais)

In [ ]:
# Exclusão das tabelas sem dados relavantes ID, SLUG, ID, ID_PAIS
df_summary_country = df_summary_dados_pais.drop('ID', 'SLUG', 'ID', 'ID_PAIS')

In [ ]:
display(df_summary_country)

In [ ]:
# Salva a tabela no formato parquet no diretório ready
df_summary_country.write.parquet("dbfs:/FileStore/_covid_data_lake/_ready/_summary/summary_country.parquet", mode='overwrite')

### Tabela data_by_country

In [ ]:
# Realizado a junção das tabelas paises e resumo
df_country_data = df_pais.join(df_dados_paises,on=df_pais.ID == df_dados_paises.ID_PAIS)
display(df_country_data)

In [ ]:
# Exclusão das tabelas sem dados relavantes ID, SLUG, ID, ID_PAIS, LAT, LON
df_data_by_country = df_country_data.drop('ID', 'SLUG', 'ID', 'ID_PAIS', 'LAT', 'LON')
display(df_data_by_country)

In [ ]:
# Salva a tabela no formato parquet no diretório ready
df_data_by_country.repartition(7).write.parquet("dbfs:/FileStore/_covid_data_lake/_ready/_data_by_country/data_by_country.parquet", mode='overwrite')